In [42]:
#r "../src/NMoneys/bin/Debug/net6.0/NMoneys.dll"

using NMoneys;
using NMoneys.Support;
using System.Numerics;
using System.Reflection;
using System.Globalization;

Func<Currency, FieldInfo> fieldOf = c => typeof(CurrencyIsoCode).GetField(c.AlphabeticCode);

In [43]:
var byIsoSymbol = CultureInfo.GetCultures(CultureTypes.AllCultures)
	.Where(ci => !ci.IsNeutralCulture && !ci.Equals(CultureInfo.InvariantCulture))
	.Select(ci => new { Culture = ci, Region = new RegionInfo(ci.Name) })
	.GroupBy(a => a.Region.ISOCurrencySymbol, StringComparer.Ordinal)
	.OrderBy(g => g.Key);


In [44]:
var canonicals = Currency.FindAll()
	.Select(c =>
	{
		CanonicalCultureAttribute attr = fieldOf(c).GetCustomAttribute<CanonicalCultureAttribute>();
		return new { Currency = c, Canonical = attr?.Culture(), Overwritten = attr?.Overwritten ?? false };
	})
	.Where(a => a.Canonical is not null)
	.ToDictionary(a => a.Currency.AlphabeticCode, a => a, StringComparer.Ordinal);

In [49]:
var results = byIsoSymbol.Aggregate(
	new { 
		NotInCurrencies = new Dictionary<string, object>(StringComparer.Ordinal), // cultures which currency is not in library
		AlternativesToCanonical = new Dictionary<string, object>(StringComparer.Ordinal)
	}, // alternative to canonical of supported currency
	(acc, group) =>
	{
		if(canonicals.TryGetValue(group.Key, out var canonical))
		{
			var alternatives = group
				.Where(a => !a.Culture.Equals(canonical.Canonical))
				.Select(a => a.Culture.Name)
				.ToArray();
			if(alternatives.Length > 0)
			{
				acc.AlternativesToCanonical.Add(group.Key,
					new { Canonical = canonical.Canonical.Name, Alternatives = alternatives }
				);
			}
		}
		else
		{
			acc.NotInCurrencies.Add(group.Key, group);
		}
		
		return acc;
	}
);

results

{ NotInCurrencies = System.Collections.Generic.Dictionary`2[System.String,System.Object], AlternativesToCanonical = System.Collections.Generic.Dictionary`2[System.String,System.Object] } NotInCurrencies key value ¤¤ [ { Culture = ar-001, Region = 001 }, { Culture = en-001, Region = 001 }, { Culture = en-150, Region = 150 }, { Culture = eo-001, Region = 001 }, { Culture = es-003, Region = 003 }, { Culture = es-419, Region = 419 }, { Culture = ia-001, Region = 001 }, { Culture = io-001, Region = 001 }, { Culture = jbo-001, Region = 001 } ] Key ¤¤ (values) index value 0 { Culture = ar-001, Region = 001 } Culture ar-001 Parent ar Parent LCID 1 KeyboardLayoutId 1 Name ar IetfLanguageTag ar DisplayName Arabic NativeName العربية EnglishName Arabic TwoLetterISOLanguageName ar ThreeLetterISOLanguageName ara ThreeLetterWindowsLanguageName ARA CompareInfo CompareInfo - ar TextInfo TextInfo - ar IsNeutralCulture True CultureTypes NeutralCultures NumberFormat System.Globalization.NumberFormatInfo DateTimeFormat System.Globalization.DateTimeFormatInfo Calendar System.Globalization.GregorianCalendar OptionalCalendars [ System.Globalization.GregorianCalendar, System.Globalization.HijriCalendar ] UseUserOverride False IsReadOnly False LCID 4096 KeyboardLayoutId 1033 Name ar-001 IetfLanguageTag ar-001 DisplayName Arabic (world) NativeName العربية (العالم) EnglishName Arabic (world) TwoLetterISOLanguageName ar ThreeLetterISOLanguageName ara ThreeLetterWindowsLanguageName ZZZ CompareInfo CompareInfo - ar-001 Name ar-001 Version System.Globalization.SortVersion LCID 4096 TextInfo TextInfo - ar-001 ANSICodePage 1256 OEMCodePage 720 MacCodePage 10004 EBCDICCodePage 20420 LCID 4096 CultureName ar-001 IsReadOnly True ListSeparator ; IsRightToLeft True IsNeutralCulture False CultureTypes SpecificCultures, UserCustomCulture NumberFormat System.Globalization.NumberFormatInfo CurrencyDecimalDigits 2 CurrencyDecimalSeparator ٫ IsReadOnly True CurrencyGroupSizes [ 3 ] NumberGroupSizes [ 3 ] PercentGroupSizes [ 3 ] CurrencyGroupSeparator ٬ CurrencySymbol ¤ NaNSymbol ليس رقم CurrencyNegativePattern 8 NumberNegativePattern 1 PercentPositivePattern 1 PercentNegativePattern 1 NegativeInfinitySymbol ؜-∞ NegativeSign ؜- NumberDecimalDigits 3 NumberDecimalSeparator ٫ NumberGroupSeparator ٬ CurrencyPositivePattern 3 PositiveInfinitySymbol ∞ PositiveSign ؜+ PercentDecimalDigits 3 PercentDecimalSeparator ٫ PercentGroupSeparator ٬ PercentSymbol ٪؜ PerMilleSymbol ؉ NativeDigits [ ٠, ١, ٢, ٣, ٤, ٥, ٦, ٧, ٨, ٩ ] DigitSubstitution Context DateTimeFormat System.Globalization.DateTimeFormatInfo AMDesignator ص Calendar System.Globalization.GregorianCalendar DateSeparator ‏/ FirstDayOfWeek Monday CalendarWeekRule FirstDay FullDateTimePattern dddd، d MMMM، yyyy h:mm:ss tt LongDatePattern dddd، d MMMM، yyyy LongTimePattern h:mm:ss tt MonthDayPattern d MMMM PMDesignator م RFC1123Pattern ddd, dd MMM yyyy HH':'mm':'ss 'GMT' ShortDatePattern d‏/M‏/yyyy ShortTimePattern h:mm tt SortableDateTimePattern yyyy'-'MM'-'dd'T'HH':'mm':'ss TimeSeparator : UniversalSortableDateTimePattern yyyy'-'MM'-'dd HH':'mm':'ss'Z' YearMonthPattern MMMM yyyy AbbreviatedDayNames [ أحد, اثنين, ثلاثاء, أربعاء, خميس, جمعة, سبت ] ShortestDayNames [ أحد, اثنين, ثلاثاء, أربعاء, خميس, جمعة, سبت ] DayNames [ الأحد, الاثنين, الثلاثاء, الأربعاء, الخميس, الجمعة, السبت ] AbbreviatedMonthNames [ يناير, فبراير, مارس, أبريل, مايو, يونيو, يوليو, أغسطس, سبتمبر, أكتوبر, نوفمبر, ديسمبر, ] MonthNames [ يناير, فبراير, مارس, أبريل, مايو, يونيو, يوليو, أغسطس, سبتمبر, أكتوبر, نوفمبر, ديسمبر, ] IsReadOnly True NativeCalendarName التقويم الميلادي AbbreviatedMonthGenitiveNames [ يناير, فبراير, مارس, أبريل, مايو, يونيو, يوليو, أغسطس, سبتمبر, أكتوبر, نوفمبر, ديسمبر, ] MonthGenitiveNames [ يناير, فبراير, مارس, أبريل, مايو, يونيو, يوليو, أغسطس, سبتمبر, أكتوبر, نوفمبر, ديسمبر, ] Calendar System.Globalization.GregorianCalendar MinSupportedDateTime 1/1/0001 12:00:00 AM MaxSupportedDateTime 12/31/9999 11:59:59 PM AlgorithmType SolarCalendar 